# FDX Stock Article extractions

In [2]:
import requests

# api_key = '23f5208b7e2940aeb8b72c67195565b2'
api_key = 'bd9bb272078b41ebb0ee0a905d4a9532'
date_from = '2023-12-11'
date_to = '2024-01-08'
# Expanded list of sources
sources = 'bloomberg,cnbc,reuters,financial-times,techcrunch,the-wall-street-journal,the-verge,business-insider,the-economist,wired,engadget,bbc-news,fortune,techradar'

# Broadening the search query
query = ('FedEx OR "FDX" OR "Frederick W. Smith" OR "Express delivery" OR "Courier services" '
         'OR "Logistics services" OR "FedEx Ground" OR "FedEx Freight" OR "Shipping industry" '
         'OR "Parcel delivery" OR "E-commerce delivery" OR "Supply chain" OR "FedEx logistics" '
         'OR "FedEx innovations" OR "Delivery technology" OR "FedEx and ecommerce" '
         'OR "FedEx sustainability" OR "Delivery drones" OR "FedEx automation"')

url = f'https://newsapi.org/v2/everything?q={query}&from={date_from}&to={date_to}&sources={sources}&apiKey={api_key}'
response = requests.get(url)
data = response.json()

# Check for 'articles' in response and extract headlines with publication dates
if 'articles' in data:
    news_data = [(article['title'], article['publishedAt'][:10]) for article in data['articles']]
    for headline, date in news_data:
        print(f"Date: {date}, Headline: {headline}")
else:
    print("Error or no articles found. Response:")
    print(data)


Date: 2024-01-08, Headline: Digitization Beats Deforestation
Date: 2023-12-15, Headline: Attacks on ships in the Red Sea are escalating, sparking fears of supply-chain disruptions
Date: 2024-01-01, Headline: Biden's EV tax credit just got a major change to help you buy a car — but way fewer options are available
Date: 2023-12-20, Headline: The wild attacks on ships in the Red Sea are another geopolitical shock that could drive up inflation
Date: 2024-01-04, Headline: The first US commercial-scale offshore wind farm is live, but the industry faces strong headwinds
Date: 2024-01-07, Headline: You can thank the Fed — not repaired supply chains — for falling inflation, think tank says
Date: 2023-12-31, Headline: 20 Things That Made the World a Better Place in 2023
Date: 2023-12-13, Headline: Scotland's spaceport plans secure funding boost
Date: 2023-12-29, Headline: Nvidia releases slower, less powerful AI chip for China
Date: 2024-01-08, Headline: Inflation could see a resurgence, and inv

# Sentiment Analysis on News Articles

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Initialize tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('yiyanghkust/finbert-tone')
model = AutoModelForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')

# Extract just the headlines from news_data
headlines = [headline for headline, _ in news_data]

# Tokenize and encode headlines
max_length = 512  # You can adjust this based on your requirements
inputs = tokenizer(headlines, padding=True, truncation=True, max_length=max_length, return_tensors="pt")

# Predict sentiment
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)

# Pair each sentiment score with its corresponding date
sentiment_data = []
for (headline, date), prediction in zip(news_data, predictions):
    sentiment_data.append({
        'date': date,
        'headline': headline,
        'sentiment_positive': prediction[0].item(),
        'sentiment_neutral': prediction[1].item(),
        'sentiment_negative': prediction[2].item()
    })


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [4]:
sentiment_data

[{'date': '2024-01-08',
  'headline': 'Digitization Beats Deforestation',
  'sentiment_positive': 0.9975757002830505,
  'sentiment_neutral': 0.0021995778661221266,
  'sentiment_negative': 0.00022473403078038245},
 {'date': '2023-12-15',
  'headline': 'Attacks on ships in the Red Sea are escalating, sparking fears of supply-chain disruptions',
  'sentiment_positive': 1.4745712178410031e-05,
  'sentiment_neutral': 1.0808357728819828e-05,
  'sentiment_negative': 0.9999744892120361},
 {'date': '2024-01-01',
  'headline': "Biden's EV tax credit just got a major change to help you buy a car — but way fewer options are available",
  'sentiment_positive': 0.9891452193260193,
  'sentiment_neutral': 0.0021334353368729353,
  'sentiment_negative': 0.008721419610083103},
 {'date': '2023-12-20',
  'headline': 'The wild attacks on ships in the Red Sea are another geopolitical shock that could drive up inflation',
  'sentiment_positive': 6.12366056884639e-06,
  'sentiment_neutral': 7.505968255827611e-

# Historical Stock Data and Sentiment Analysis Combined

In [5]:
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import StandardScaler

# Convert sentiment_data to a DataFrame
sentiment_df = pd.DataFrame(sentiment_data)

# Convert date to datetime and set as index
sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])
sentiment_df.set_index('date', inplace=True)

# Aggregate sentiment scores by date
average_sentiment = sentiment_df.groupby('date').mean()

# Download stock data
ticker_symbol = 'FDX'
stock_data = yf.download(ticker_symbol, start='2023-12-09', end='2024-01-08')

# Selecting the required columns
stock_data = stock_data[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]

# Convert stock_data index to datetime
stock_data.index = pd.to_datetime(stock_data.index)

# Combine stock data with sentiment data
combined_data = stock_data.join(average_sentiment)

# Interpolate missing values
combined_data = combined_data.interpolate(method='time')

# Scale price-related columns
price_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close']
scaler_prices = StandardScaler()
combined_data[price_cols] = scaler_prices.fit_transform(combined_data[price_cols])

# Scale Volume column independently
scaler_volume = StandardScaler()
combined_data['Volume'] = scaler_volume.fit_transform(combined_data[['Volume']])

print(combined_data)

<ipython-input-5-a43c3141ee83>:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  average_sentiment = sentiment_df.groupby('date').mean()


[*********************100%%**********************]  1 of 1 completed
                Open      High       Low     Close  Adj Close    Volume  \
Date                                                                      
2023-12-11  0.868606  0.780532  0.942747  0.796355   0.796355 -0.651758   
2023-12-12  0.979339  0.944688  1.019745  0.912686   0.912686 -0.588845   
2023-12-13  0.874436  0.848290  0.757220  0.971206   0.971206 -0.599116   
2023-12-14  1.272192  1.363108  1.300601  1.373012   1.373012 -0.241380   
2023-12-15  1.297690  1.522373  1.368066  1.503618   1.503618 -0.273042   
2023-12-18  1.691076  1.592926  1.643056  1.548580   1.548580 -0.349650   
2023-12-19  1.532265  1.456711  1.486128  1.411552   1.411552  0.785879   
2023-12-20 -0.986137 -0.701754 -0.902253 -0.997138  -0.997138  3.858492   
2023-12-21 -0.868850 -0.976276 -0.997582 -0.917919  -0.917919  0.521097   
2023-12-22 -0.933686 -0.929475 -0.908119 -0.870102  -0.870102 -0.124665   
2023-12-26 -0.731165 -0.659143 

In [6]:
import pandas as pd
df = pd.DataFrame(combined_data)
df

,Open,High,Low,Close,Adj Close,Volume,sentiment_positive,sentiment_neutral,sentiment_negative
Date,,,,,,,,,
2023-12-11,0.868606,0.780532,0.942747,0.796355,0.796355,-0.651758,0.998814,0.000091,0.001095
2023-12-12,0.979339,0.944688,1.019745,0.912686,0.912686,-0.588845,0.746141,0.000168,0.253691
2023-12-13,0.874436,0.848290,0.757220,0.971206,0.971206,-0.599116,0.499706,0.500001,0.000294
2023-12-14,1.272192,1.363108,1.300601,1.373012,1.373012,-0.241380,0.999981,0.000001,0.000018
2023-12-15,1.297690,1.522373,1.368066,1.503618,1.503618,-0.273042,0.192225,0.307331,0.500444
2023-12-18,1.691076,1.592926,1.643056,1.548580,1.548580,-0.349650,0.133642,0.615669,0.250689
2023-12-19,1.532265,1.456711,1.486128,1.411552,1.411552,0.785879,0.458560,0.002237,0.539203
2023-12-20,-0.986137,-0.701754,-0.902253,-0.997138,-0.997138,3.858492,0.333215,0.000156,0.666629
2023-12-21,-0.868850,-0.976276,-0.997582,-0.917919,-0.917919,0.521097,0.165268,0.000098,0.834634


# Transformer Architecture

In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
from tensorflow.keras.callbacks import EarlyStopping

selected_features = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sentiment_positive', 'sentiment_neutral', 'sentiment_negative']
data = combined_data[selected_features]

train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.15)
train_data = data.iloc[:train_size]
val_data = data.iloc[train_size:train_size + val_size]
test_data = data.iloc[train_size + val_size:]

# Prepare features and target for model training
X_train = train_data.drop('Close', axis=1)
y_train = train_data['Close']
X_val = val_data.drop('Close', axis=1)
y_val = val_data['Close']
X_test = test_data.drop('Close', axis=1)
y_test = test_data['Close']

# Define and compile the neural network model
model = Sequential()
model.add(Dense(32, activation='LeakyReLU', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(16, activation='LeakyReLU'))
model.add(Dense(1))
model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(
    X_train, y_train,
    epochs=50,  # Set back to 50 or an arbitrarily large number
    batch_size=8,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]  # Add the early stopping callback here
)

train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(train_loss) + 1)

# Convert the epochs range object to a list
epochs_list = list(epochs)

# Plotting the training and validation loss
loss_fig = go.Figure()
loss_fig.add_trace(go.Scatter(x=epochs_list, y=train_loss, mode='lines', name='Training Loss'))
loss_fig.add_trace(go.Scatter(x=epochs_list, y=val_loss, mode='lines', name='Validation Loss'))
loss_fig.update_layout(title='Training and Validation Loss per Epoch',
                       xaxis_title='Epochs',
                       yaxis_title='Loss',
                       legend_title='Type')
loss_fig.show()

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print(f'MSE: {mse}, MAE: {mae}, R^2: {r2}')

# Reshape y_test and predictions for inverse transform
y_test_reshaped = y_test.values.reshape(-1, 1)
predictions_reshaped = predictions.reshape(-1, 1)

num_scaled_cols = 5

# Create separate dummy arrays for inverse scaling
dummy_array_y_test = np.zeros((len(y_test_reshaped), num_scaled_cols))
dummy_array_predictions = np.zeros((len(predictions_reshaped), num_scaled_cols))

# Fill in the 'Close' column values in the dummy arrays
# Assuming 'Close' is the last of the scaled columns
dummy_array_y_test[:, -1] = y_test_reshaped.flatten()
dummy_array_predictions[:, -1] = predictions_reshaped.flatten()

# Inverse transform the 'Close' prices using the dummy arrays
y_test_original = scaler_prices.inverse_transform(dummy_array_y_test)[:, -1]
predictions_original = scaler_prices.inverse_transform(dummy_array_predictions)[:, -1]

# Extracting testing dates
test_dates = test_data.index

# Plotting with Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_dates, y=y_test_original, mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=test_dates, y=predictions_original, mode='lines', name='Predicted'))
fig.update_layout(title='Actual vs Predicted Stock Prices', xaxis_title='Date', yaxis_title='Stock Price', legend_title='Legend')
fig.show()

Epoch 1/50
2/2 [==============================] - 1s 124ms/step - loss: 1.4343 - val_loss: 0.0401
Epoch 2/50
2/2 [==============================] - 0s 30ms/step - loss: 0.1768 - val_loss: 0.0257
Epoch 3/50
2/2 [==============================] - 0s 27ms/step - loss: 0.1590 - val_loss: 0.1021
Epoch 4/50
2/2 [==============================] - 0s 32ms/step - loss: 0.5040 - val_loss: 0.0644
Epoch 5/50
2/2 [==============================] - 0s 31ms/step - loss: 0.4514 - val_loss: 0.0202
Epoch 6/50
2/2 [==============================] - 0s 33ms/step - loss: 0.2797 - val_loss: 0.0041
Epoch 7/50
2/2 [==============================] - 0s 30ms/step - loss: 0.1104 - val_loss: 0.0162
Epoch 8/50
2/2 [==============================] - 0s 29ms/step - loss: 0.0944 - val_loss: 0.0370
Epoch 9/50
2/2 [==============================] - 0s 33ms/step - loss: 0.0744 - val_loss: 0.0521
Epoch 10/50
2/2 [==============================] - 0s 25ms/step - loss: 0.0537 - val_loss: 0.0628
Epoch 11/50
2/2 [===========

1/1 [==============================] - 0s 48ms/step
MSE: 0.002253825524796173, MAE: 0.03776206745328384, R^2: 0.9341219652158961


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 833 (3.25 KB)
Trainable params: 833 (3.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
